<a href="https://colab.research.google.com/github/francji1/01ZLMA/blob/main/R/01ZLMA_ex10_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01ZLMA - Exercise 10
Exercise 10 of the course 01ZLMA. 

## Contents

* Log-linear models with Poisson distributed data
 ---
* Example from the Lecture 10
 ---
* ..



Budeme uvažovat data z příkladu v Sekci 7.7 obsahující údaje o počtech úmrtí na cévně-srdeční onemocnění v závislosti na věku a kouření.

In [ ]:
data_poiss <- "https://raw.githubusercontent.com/francji1/01ZLMA/main/data/smoke.csv"
data_poiss  <- read.table(data_poiss, header = T, sep = ";")
data_poiss

In [ ]:
library(tidyverse)

In [ ]:

data_poiss <- data_poiss %>%
 add_column(age_avg = c(40,50,60,70,80,40,50,60,70,80),
            age_min = c(35,45,55,65,75,35,45,55,65,75)) %>%
 mutate(n_min = ceiling(Person.years/(age_min-10)),
        n_avg = ceiling(Person.years/(age_avg-10)),
        Living = n_min - Deaths)
 str(data_poiss)
 data_poiss

In [ ]:
assign(data_poiss)

Ověříme, zda je proměnná Smoke faktor a vytvoříme proměnnou agecat

In [ ]:
is.factor(Smoke)
Smoke <- as.factor(Smoke)

is.factor(Age.group)
agecat <- as.numeric(Age.group)
 agecat

## Nejjednodušší aditivní model 
Začneme nejjednodušším modelem s proměnnými `Smoke` a `agecat`

In [ ]:
mdl <- glm( Deaths ~ offset(log(Person.years)) + Smoke + agecat, family=poisson(link="log"))
summary(mdl)

Deviační statistika má obrovskou hodnotu a model tak moc neodpovídá datům.
Zobrazíme závislost logaritmu škálovaných hodnot Y na proměnné agecat

In [ ]:
y.scaled <- Deaths/Person.years*100000
plot(agecat, log(y.scaled), pch = c(15:16)[Smoke], col = c("red","blue")[Smoke], 
     cex=1.2, xlab = "agecat", ylab = "počty úmrtí")

Je jasně vidět, že vztah není lineární, přidáme do modelu proměnnou `agecat^2`

## Model s kvadratickou závislostí na věku - Model 0  

In [ ]:
mdl.0 <- glm( Deaths ~ offset(log(Person.years)) + Smoke + agecat + I(agecat^2), 
              family=poisson(link="log"))
summary(mdl.0)

Všechny regresory jsou v modelu významné a hodnota deviační statistiky klesla na 12.176, porovnejme ji s kritickou hodnotou LRT testu

In [ ]:
Xm <- model.matrix(mdl.0)
n<-nrow(Xm); p<-ncol(Xm); n; p
# kritická hodnota
c.val <- qchisq(0.05, n-p, ncp=0, lower.tail = FALSE); c.val
# p-hodnota testu
p.val<-pchisq(deviance(mdl.0), n-p, ncp=0, lower.tail = FALSE); p.val

Hypotézu, že model dobře popisuje data jsme na hladině významnosti 5% těsně nezamítli. Zkusíme ještě přidat do modelu interakci (na obrázku z přednášky byla vidět možná změna závislosti na kouření s rostoucím věkem)

## Model s kvadratickou závislostí na věku a interakcí - Model 1 

In [ ]:
mdl.1 <- glm( Deaths ~ offset(log(Person.years)) + Smoke + agecat + I(agecat^2) + Smoke:agecat, 
              family=poisson(link="log"))
summary(mdl.1)

Všechny proměnné jsou v modelu významné a hodnota deviační statsitiky klesla na hodnotu 1.6354. 
Pro kritickou hodnotu LRT testu dostaneme

In [ ]:
Xm <- model.matrix(mdl.1)
n<-nrow(Xm); p<-ncol(Xm); n; p
# kritická hodnota
c.val <- qchisq(0.05, n-p, ncp=0, lower.tail = FALSE); c.val
# p-hodnota testu
p.val<-pchisq(deviance(mdl.1), n-p, ncp=0, lower.tail = FALSE); p.val

a hypotézu vhodnosti modelu tedy nezamítáme s p-hodnotou 0.897. To indikuje, že model dobře popisuje data.

Pro ilustraci ještě spočtěme hodnotu deviační statistiky pomocí vzorce z přednášky

In [ ]:
y <- Deaths
mu.hat <- fitted(mdl.1)               # predikované hodnoty modelem 1
dev <- y*log(y/mu.hat) - (y - mu.hat)
dev.stat <- 2*sum(dev); dev.stat

Dostáváme stejnou hodnotu jako pomocí funkce `glm()`.

Ještě otestujeme významnost interakce pomocí vzorce odvozeného na přednášce

In [ ]:
mu.tilde <- fitted(mdl.0)            # predikované hodnoty modelem 0
dev <- y * log(mu.hat/mu.tilde) - mu.hat + mu.tilde
dev.stat <- 2*sum(dev); dev.stat     # deviační statistika
deviance(mdl.0) - deviance(mdl.1)    # pro kontrolu pomocí naladěných modelů

pro kritickou hodnotu testu platí 

In [ ]:
# kritická hodnota
c.val <- qchisq(0.05, 1, ncp=0, lower.tail = FALSE); c.val
# p-hodnota testu
p.val<-pchisq(dev.stat, 1, ncp=0, lower.tail = FALSE); p.val

interakce je tedy v modelu významná. Případně můžeme použít přímo funkci `anova()`

In [ ]:
anova(mdl.0,mdl.1, test="Chisq")

která vrací stejný výsledek.


## Analýza Modelu 1 {#anl}

Scatterplot pro pozorované a predikované hodnoty vysvětlované proměnné

In [ ]:
plot(y,mu.hat,col=2, cex=1.0, lwd=2)
abline(1,1,col=4,lwd=2)

Pozorované a predikované hodnoty velmi dobře odpovídají.

**Obrázky reziduí**

In [ ]:
par(mfrow=c(2,2))

r.ps <- rstandard(mdl.1, type="pearson") # standardizovaná Pearsonova rezidua

# rezidua vs. fitted values
plot(mu.hat,r.ps,col="red", cex=1.3, lwd=2)
abline(0,0)
# rezizua vs. agecat
plot(agecat,r.ps,col="red", cex=1.3, lwd=2)
abline(0,0)
# rezizua vs. smoke
plot(Smoke,r.ps,col="red", cex=1.3, xlab="Smoke", lwd=2)
abline(0,0)

qqnorm(r.ps)   # QQ - plot reziduí
qqline(r.ps)

vzhledem k mlému počtu pozorování zde není vidět žádný zásadní problé. Otestujme ještě formálně normalitu rezidí

In [ ]:
shapiro.test(r.ps)  # Test normality

Hypotéza normality není zamítnuta.

**Influenční pozorování a pákové body**

In [ ]:
par(mfrow=c(1,2))
c.d <- cooks.distance(mdl.1)  # Cookova vzdálenost
lev <- hatvalues(mdl.1)       # diag. prvky projekční matice

plot(c.d,col="red", cex=1.3, lwd=2, ylim = c(0,2.1))
# abline(8/(n-2*p),0)         # v našem případě máme malý počet pozorování a n-2p = 0
                              # kritická hodnota by tedy byla velká

plot(lev,col="red", cex=1.3, lwd=2, ylim = c(0,1.2))
abline(2*p/n,0)

Ani v jednom případě nevidíme podezřele velké hodnoty

## Interpretace parametrů {#RR}

Spočteme relativní rizika v Modelu 1

In [ ]:
par.est.1 <- coef(mdl.1); par.est.1
RR.1 <- exp(par.est.1); RR.1

Diskuze obdržených hodnot a výpočet relativních rizik pro jednotlivé kategorie věku jsou v Sekci 7.7 materiálů k přednášce.

Pro Model 0 bez interakcí je situace jednodušší a lze snado získat odhady RR a příslušné intervaly spolehlivosti

In [ ]:
exp(cbind(coef(mdl.0), confint(mdl.0)))

**Na závěr ještě vykresleme obrázky Modelů 0 a 1**

In [ ]:
par(mfrow=c(2,1))

x.pop<-Age.group[1:5]
s <- Person.years
y.scaled <- y/s*100000
Smoke <- ordered(Smoke, levels = c("1","0"))   # udáme pořadí úrovní faktoru Smoke, `R` je jinak prohodí, což může způsobit špatnou legendu v grafech

### Obrázek model 0
mu.tilde.scaled <- mu.tilde/s*100000

plot(agecat, y.scaled, pch = c(15:16)[Smoke],  xaxt = "n", main = "Model 0 bez interakce", col = c("red","blue")[Smoke], cex=1.0, xlab = "Age group", ylab = "Počty úmrtí")
axis(1, at=agecat[1:5], labels=x.pop)
legend("topleft", inset = .05, bty="n", legend = c("Smokers", "Non-smokers"), pch = c(15:16), cex=1.0, pt.cex = 1.0, col = c("red","blue"))
lines(agecat[1:5], mu.tilde.scaled[1:5], col="red", lwd=2 )
lines(agecat[6:10], mu.tilde.scaled[6:10], col = "blue", lwd=2 )


### Obrázek model 1
mu.hat.scaled <- mu.hat/s*100000

plot(agecat, y.scaled, pch = c(15:16)[Smoke],  xaxt = "n", main = "Model 1 s interakcí", col = c("red","blue")[Smoke], cex=1.0, xlab = "Age group", ylab = "Počty úmrtí")
axis(1, at=agecat[1:5], labels=x.pop)
legend("topleft", inset = .05, bty="n", legend = c("Smokers", "Non-smokers"), pch = c(15:16), cex=1.0, pt.cex = 1.0, col = c("red","blue"))
lines(agecat[1:5], mu.hat.scaled[1:5], col="red", lwd=2 )
lines(agecat[6:10], mu.hat.scaled[6:10], col = "blue", lwd=2 )
